<a href="https://colab.research.google.com/github/Aryannovice/JustARAG/blob/main/RAGFinal(Yardstick).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kindly load the .env file containing OPEN_AI API key , PINE_CONE as well as environment , i have attached pinecone_env and pinecone for ease of use .


PINECONE_API_KEY=pcsk_4UKn4c_Qn41pekY7VfyypYGcV6sMgHAgXpJLfPbABTgxGfGzuRWGYbqFyQgjo11tqo1LPz

PINECONE_ENV=us-east-1-aws

Also, you can upload pdf (buisness based) of whatsoever size ..
The pdf i used was NASDAQ_AAPL_2023

In [ ]:
!pip install -qU langchain langchain-openai langchain-pinecone pinecone openai python-dotenv langchain_community pyPDF sentence-transformers scikit-learn hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

imported all necessary modules to begin with

In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain.agents import create_react_agent, AgentExecutor
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import BaseTool, tool
from langchain.agents.output_parsers.react_single_input import ReActSingleInputOutputParser
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import AgentType, initialize_agent
import os
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.react.base import DocstoreExplorer
from langchain.agents import AgentType, initialize_agent
from langchain.tools import BaseTool, tool
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from sentence_transformers import CrossEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

As part of a rag , we should split the doc in such a way that extra spaces , chars needless are eliminated and thats the job of this function.

In [ ]:
def load_and_chunk(filepath: str,
                   chunk_size: int = 1000,
                   overlap: int = 200) -> list:
    """Load PDF/TXT and split into semantic chunks."""
    loader = PyPDFLoader(filepath) if filepath.endswith(".pdf") else TextLoader(filepath)
    docs = loader.load()
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", "!", "?", ",", " "]
    )
    return splitter.split_documents(docs)

Next we Initialize and configure Pinecone vector database for business document storage.

In [ ]:
import pinecone
def init_pinecone(index_name: str = "rag-business-qa",
                  dimension: int = 1536):
    pc = Pinecone(
        api_key=os.environ["PINECONE_API_KEY"]
    )

    index = pc.Index("rag-business-qa")
    # if index_name not in pc.list_indexes().names():
    #     pc.create_index(
    #         name=index_name,
    #         dimension=dimension,
    #         metric="cosine",
    #         spec=ServerlessSpec(cloud="aws", region="us-east-1")
    #    )
    if "rag-business-qa" not in [i["name"] for i in pc.list_indexes()]:
      pc.create_index(
        name="rag-business-qa",
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
                                  openai_api_key=os.environ["OPENAI_API_KEY"])
    return PineconeVectorStore(
        index=index,
        embedding=embeddings,
        )

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",
                              openai_api_key=os.environ["OPENAI_API_KEY"])



vectorstore = PineconeVectorStore(
    index_name="rag-business-qa",
    embedding=embeddings,
    pinecone_api_key=os.environ["PINECONE_API_KEY"],
    namespace=None
)


Based upon pinecone index , the names can be changed / varied.

In [ ]:
vectorstore = init_pinecone(index_name="rag-business-qa", dimension=1536)

Next we Process and store document chunks in Pinecone vector database using batch processing. I did this since pinecone has a limit of 2mb for free tier but buisness data can be big and when its about finance a little detail may be worth noting. Therefore , used batch processing for even bigger pdfs.

In [ ]:
from langchain_core.documents import Document
def ingest_chunks_in_batches(vector_store: PineconeVectorStore, chunks: list, batch_size: int = 5):
    """
    Ingest chunks in small batches to avoid Pinecone's 2MB limit.

    """
    from langchain_core.documents import Document
    import time

    total_chunks = len(chunks)
    total_batches = (total_chunks + batch_size - 1) // batch_size

    print(f"Processing {total_chunks} chunks in {total_batches} batches of {batch_size} chunks each...")

    for i in range(0, total_chunks, batch_size):
        batch_num = i // batch_size + 1
        batch = chunks[i:i + batch_size]

        print(f"Processing batch {batch_num}/{total_batches} ({len(batch)} chunks)...")

        # Convert batch to Document objects
        docs = [
            Document(
                page_content=chunk.page_content,
                metadata=chunk.metadata
            ) for chunk in batch
        ]

        try:

            vector_store.add_documents(docs)
            print(f"✓ Batch {batch_num} successfully added")


            time.sleep(0.5)

        except Exception as e:
            print(f"✗ Error in batch {batch_num}: {str(e)}")


            if "Request size" in str(e) and "exceeds" in str(e):
                print(f"  → Retrying batch {batch_num} with smaller chunks...")

                ingest_chunks_in_batches(vector_store, batch, batch_size // 2)
            else:
                print(f"  → Skipping batch {batch_num} due to error")
                continue

    print("✓ All batches processed!")


def load_and_chunk_with_monitoring(file_path: str, chunk_size: int = 500, chunk_overlap: int = 50):
    """
    Load and chunk documents with size monitoring to prevent oversized chunks.
    """
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain_community.document_loaders import PyPDFLoader
    import sys

    # Load document
    loader = PyPDFLoader(file_path)
    pages = loader.load()


    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""],
        length_function=len
    )


    chunks = text_splitter.split_documents(pages)


    total_size = 0
    oversized_chunks = 0

    for i, chunk in enumerate(chunks):
        chunk_size_bytes = len(chunk.page_content.encode('utf-8'))
        total_size += chunk_size_bytes

        if chunk_size_bytes > 100000:
            print(f"Warning: Chunk {i} is {chunk_size_bytes/1024:.1f}KB (very large)")
            oversized_chunks += 1

    print(f"Total chunks: {len(chunks)}")
    print(f"Total size: {total_size/1024/1024:.2f}MB")
    print(f"Average chunk size: {total_size/len(chunks)/1024:.1f}KB")
    print(f"Oversized chunks: {oversized_chunks}")

    return chunks


def process_large_document(file_path: str, vector_store: PineconeVectorStore):
    """
    Complete pipeline for processing large documents efficiently.
    """
    print(f"Processing large document: {file_path}")


    chunks = load_and_chunk_with_monitoring(file_path, chunk_size=300, chunk_overlap=30)


    ingest_chunks_in_batches(vector_store, chunks, batch_size=3)

    print("✓ Document processing complete!")

# Example usage:
process_large_document("/content/NASDAQ_AAPL_2023.pdf", vectorstore)

Processing large document: /content/NASDAQ_AAPL_2023.pdf
Total chunks: 1399
Total size: 0.26MB
Average chunk size: 0.2KB
Oversized chunks: 0
Processing 1399 chunks in 467 batches of 3 chunks each...
Processing batch 1/467 (3 chunks)...
✓ Batch 1 successfully added
Processing batch 2/467 (3 chunks)...
✓ Batch 2 successfully added
Processing batch 3/467 (3 chunks)...
✓ Batch 3 successfully added
Processing batch 4/467 (3 chunks)...
✓ Batch 4 successfully added
Processing batch 5/467 (3 chunks)...
✓ Batch 5 successfully added
Processing batch 6/467 (3 chunks)...
✓ Batch 6 successfully added
Processing batch 7/467 (3 chunks)...
✓ Batch 7 successfully added
Processing batch 8/467 (3 chunks)...
✓ Batch 8 successfully added
Processing batch 9/467 (3 chunks)...
✓ Batch 9 successfully added
Processing batch 10/467 (3 chunks)...
✓ Batch 10 successfully added
Processing batch 11/467 (3 chunks)...
✓ Batch 11 successfully added
Processing batch 12/467 (3 chunks)...
✓ Batch 12 successfully added
Pro

Advanced hybrid search system combining dense vector search with sparse keyword
    search and cross-encoder reranking for business document retrieval as well ReACT as a smarter approach.

In [ ]:


class BusinessHybridSearch:
    def __init__(self, vector_store):
        self.vector_store = vector_store
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
        self.tfidf = TfidfVectorizer(max_features=1000, stop_words='english')

    def business_hybrid_search(self, query: str, k: int = 5):
        """Hybrid search optimized for business queries."""


        dense_results = self.vector_store.similarity_search(query, k=k*2)


        documents = [doc.page_content for doc in dense_results]
        tfidf_matrix = self.tfidf.fit_transform(documents)
        query_vector = self.tfidf.transform([query])


        similarities = np.dot(tfidf_matrix, query_vector.T).toarray().flatten()
        sparse_indices = np.argsort(similarities)[::-1][:k]


        combined_docs = []
        for i in sparse_indices:
            combined_docs.append(dense_results[i])


        pairs = [(query, doc.page_content) for doc in combined_docs]
        scores = self.reranker.predict(pairs)


        doc_score_pairs = list(zip(scores, combined_docs))
        doc_score_pairs.sort(key=lambda x: x[0], reverse=True)
        ranked_docs = [doc for score, doc in doc_score_pairs]

        return ranked_docs[:k]

@tool
def retrieve_hybrid(query: str, k: int = 5) -> list:
    """Retrieve documents using hybrid search optimized for business queries."""
    hybrid_searcher = BusinessHybridSearch(vectorstore)
    return hybrid_searcher.business_hybrid_search(query, k)

@tool
def generate_answer(query: str) -> str:
    """Retrieve relevant documents and generate an answer."""

    docs = retrieve_hybrid(query)


    llm = ChatOpenAI(model="gpt-3.5-turbo",
                     openai_api_key=os.environ["OPENAI_API_KEY"],
                     temperature=0.4)
    prompt = PromptTemplate.from_template(
        """You are a business QA assistant. Use the context to answer.
Context:
{context}

Question: {query}

Answer concisely, citing context sources."""
    )
    context = "\n\n".join(d.page_content for d in docs)

    response = llm.invoke(prompt.format(context=context, query=query))
    return response.content


tools: list[BaseTool] = [retrieve_hybrid, generate_answer]


react_prompt = PromptTemplate.from_template(
    """You are a business QA assistant. Answer questions using the available tools.

Available tools:
{tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought: {agent_scratchpad}"""
)


agent = create_react_agent(
    llm=ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"], temperature=0.3),
    tools=tools,
    prompt=react_prompt,
    output_parser=ReActSingleInputOutputParser(),
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True  )

result = agent_executor.invoke({"input": "What is our revenue trend?", "agent_scratchpad": ""})
print(result)



> Entering new AgentExecutor chain...
I should retrieve documents related to our revenue trend to provide an accurate answer.
Action: retrieve_hybrid
Action Input: "revenue trend"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

[Document(id='b0f13b7e-0d90-40b0-8cd5-1b6ae985aa94', metadata={'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'creationdate': '2023-11-03T06:02:32-04:00', 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320193-23-000106; ; 10-K', 'moddate': '2023-11-03T06:02:40-04:00', 'page': 36.0, 'page_label': '37', 'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'source': '/content/NASDAQ_AAPL_2023.pdf', 'subject': 'Form 10-K filed on 2023-11-03 for the period ending 2023-09-30', 'title': '0000320193-23-000106', 'total_pages': 80.0}, page_content='Note 2 – Revenue\nThe Company recognizes revenue at the amount to which it expects to be entitled when control of the products or services is transferred to its customers.'), Document(id='da844a85-d1b0-480d-8940-d557b8242e1a', metadata={'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'creationdate': '2023-11-03T06:02:32-04:00', 'creator': 'EDGAR Filing HTML Converter', 'keywords': '0000320193-23-000106; ;

/tmp/ipython-input-9-2811614279.py:48: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retrieve_hybrid(query)


Based on the historical trend mentioned in the context, the Company typically experiences higher net sales in the first quarter compared to other quarters due to seasonal holiday demand and new product introductions.I now know the final answer.
Final Answer: Based on the historical trend mentioned in the financial report, the Company typically experiences higher net sales in the first quarter compared to other quarters due to seasonal holiday demand and new product introductions.

> Finished chain.
{'input': 'What is our revenue trend?', 'agent_scratchpad': '', 'output': 'Based on the historical trend mentioned in the financial report, the Company typically experiences higher net sales in the first quarter compared to other quarters due to seasonal holiday demand and new product introductions.'}


This isa testing based section for the same , batches might be little less from previous ...

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

vectorstore = init_pinecone()

chunks = load_and_chunk("/content/NASDAQ_AAPL_2023.pdf")
ingest_chunks_in_batches(vectorstore, chunks, batch_size=3)

def ask(question: str) -> str:
    result = agent_executor.invoke({"input": question})
    return result["output"]

print(ask("What were the main revenue drivers last quarter?"))

Processing 362 chunks in 121 batches of 3 chunks each...
Processing batch 1/121 (3 chunks)...
✓ Batch 1 successfully added
Processing batch 2/121 (3 chunks)...
✓ Batch 2 successfully added
Processing batch 3/121 (3 chunks)...
✓ Batch 3 successfully added
Processing batch 4/121 (3 chunks)...
✓ Batch 4 successfully added
Processing batch 5/121 (3 chunks)...
✓ Batch 5 successfully added
Processing batch 6/121 (3 chunks)...
✓ Batch 6 successfully added
Processing batch 7/121 (3 chunks)...
✓ Batch 7 successfully added
Processing batch 8/121 (3 chunks)...
✓ Batch 8 successfully added
Processing batch 9/121 (3 chunks)...
✓ Batch 9 successfully added
Processing batch 10/121 (3 chunks)...
✓ Batch 10 successfully added
Processing batch 11/121 (3 chunks)...
✓ Batch 11 successfully added
Processing batch 12/121 (3 chunks)...
✓ Batch 12 successfully added
Processing batch 13/121 (3 chunks)...
✓ Batch 13 successfully added
Processing batch 14/121 (3 chunks)...
✓ Batch 14 successfully added
Processin